# Physics 494/594
## Gradient of a Neural Network

In [ ]:
# %load ./include/header.py
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import trange,tqdm
sys.path.append('./include')
import ml4s
import jax.numpy as jnp 
from jax import grad
from jax import random

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('./include/notebook.mplstyle')
np.set_printoptions(linewidth=120)
ml4s.set_css_style('./include/bootstrap.css')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
π = np.pi

## Last Time

### [Notebook Link: 14_Stochastic_Gradient_Descent.ipynb](./14_Stochastic_Gradient_Descent.ipynb)

- Use randomness to effeciently optimize a rough convex function
- Minibatches (data subsets) and epochs (all possible minibatches)

## Today

- Taking the gradient of a neural network.

In lecture, we manually determined the gradient of the cost function of a shallow neural network with 2 input neurons and one output.

In [ ]:
ml4s.draw_network([2,1], zero_index=False )

## Cost & Gradient

For a data set: $\mathcal{D} = \left\{\vec{x}^{(n)},y^{(n)}\right\}_{n=1}^{N}$, we wrote the cost function as the usual least squared error:

\begin{equation}
C\left(w_{11}^1,w_{21}^1,b_1^1; \mathcal{D}\right) = \frac{1}{2} \langle \left(a_1^1 - y\right)^2 \rangle
\end{equation}

where $\langle \dots \rangle$ indicates an average over $\mathcal{D}$.  By using the chain rule we found:

\begin{align}
\frac{\partial C}{\partial w_{j1}^1} &= \langle \left(a_1^1 - y\right)f^\prime\left(z_1^1\right)x_j  \rangle \\
\frac{\partial C}{\partial b_{1}^1} &= \langle \left(a_1^1 - y\right)f^\prime\left(z_1^1\right) \rangle \, .
\end{align}

Let's assume our non-linearity is a sigmoid:

\begin{equation}
f(z) = \sigma(z) = \frac{1}{1+e^{-z}}
\end{equation}

such that the derivative will be:
\begin{equation}
f^\prime(z) = \frac{e^{-z}}{\left(1+e^{-z}\right)^2}.
\end{equation}

I will use `jax.numpy` for these functions as a drop-in replacement for `numpy`.

In [ ]:
def σ(z):
    return 1.0/(1.0 + jnp.exp(-z))

def σ_prime(z):
    return jnp.exp(-z)/(1.0 + jnp.exp(-z))**2

## Feed Forward

The prediction of our network is a sipmle 1-line funciton (we have a shallow network with no hidden layers).

In [ ]:
def feed_forward(w, b, x):
    return σ(jnp.dot(x, w) + b)

## A synthetic data set

We will just check the accuracy of our derivative method, so we generate a random (synthetic) data set.

In [ ]:
batchsize = 10
x = np.random.uniform(low=-1,high=1,size=[batchsize,2])
y = np.random.uniform(low=-2,high=2,size=[batchsize,])

## Initializing the Network

As we have done in the past, let's start with some random weights and biases.

In [ ]:
w = np.random.uniform(low=-1,high=1,size=[2])
b = np.random.uniform(low=-1,high=1,size=1)

## The cost function

\begin{equation}
C\left(w_{11}^1,w_{21}^1,b_1^1; \mathcal{D}\right) = \frac{1}{2} \langle \left(a_1^1 - y\right)^2 \rangle
\end{equation}

In [ ]:
z1 = jnp.dot(x, w) + b
a1 = σ(z1)
print(a1)

In [ ]:
def cost(w,b,x,y):
    a1 = feed_forward(w,b,x)
    return 0.5*jnp.average((a1-y)**2)

In [ ]:
cost(w,b,x,y)

## Gradient of the cost function

\begin{align}
\frac{\partial C}{\partial w_{j1}^1} &= \langle \left(a_1^1 - y\right)f^\prime\left(z_1^1\right)x_j  \rangle \\
\frac{\partial C}{\partial b_{1}^1} &= \langle \left(a_1^1 - y\right)f^\prime\left(z_1^1\right) \rangle \, .
\end{align}

In [ ]:
def dC_dw(w,b,x,y):
    z1 = jnp.dot(x, w) + b
    a1 = σ(z1)
    fp = σ_prime(z1)
    return jnp.dot((a1-y)*fp, x) / x.shape[0]

def dC_db(w,b,x,y):
    z1 = jnp.dot(x, w) + b
    a1 = σ(z1)
    fp = σ_prime(z1)
    return jnp.average((a1-y)*fp)

### Investigate the output

In [ ]:
dCw = dC_dw(w,b,x,y)
ml4s.mdtex(rf'$\nabla_w C = [{dCw[0]:.6f},\, {dCw[1]:.6f}]$')
dCb = dC_db(w,b,x,y)
ml4s.mdtex(rf'$\nabla_b C = {dCb:.6f}$')

### Check that these gradients will allow us to step down hill

In [ ]:
print(f'cost before: {cost(w,b,x,y):.6f}')
η = 0.9
w -= η*dC_dw(w,b,x,y)
b -= η*dC_db(w,b,x,y)
print(f'cost after: {cost(w,b,x,y):.6f}')

### It Works!

---

<div class="span alert alert-success">
<h2> Programming Excercise </h2>
    
<ol>
    <li>Check that our hand-coded gradients are correct using <code>jax</code>.
    <li> Iterate over <code>num_iter = 100</code> simple gradient descent steps and plot the resulting cost vs. iteration. What do you see?
    </li>
</ol>
</div>

In [ ]:
gradwC = # INSERT CODE HERE
gradbC = # INSERT CODE HERE

### Compare the results

In [ ]:
# INSERT CODE HERE

### Track the reduction in cost

In [ ]:
η = 0.9
num_iter = 100

C = []
for n in range(num_iter):
    # INSERT CODE HERE
    pass

### Plot the result

In [ ]:
plt.plot(C)
plt.xlabel('Iteration Step')
plt.ylabel('Cost')